# Pipeline

In [1]:
%load_ext autoreload
%autoreload 2

## Torch Check

In [2]:
import numpy as np
import torch
from feature_detection import Net
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from _datasets import ObjectPointCloudDataset

## Model Instantiation

In [3]:
model = Net()
model.to('cuda')
model.load_state_dict(torch.load('models/modelv9.pt')['model_state_dict'])
model.eval()

def get_feature_vector(pc):
    # Creating the data object
    data = Data(x = torch.tensor(pc, dtype=torch.float32)) 
    loader = DataLoader(dataset=[data], batch_size=1)

    # Obtaining the prediction
    for d in loader:
        d = d.to('cuda')
        result = model(d)[1]

    return result.cpu().detach().numpy()

## Mujoco Simulation

In [4]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import os
from agent import Agent

# Mujoco Libraries
import mujoco
import mujoco.viewer
from sensors import Sensors
from vectorized_point_cloud import VectorizedPC

import open3d as o3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Beginning Simulation on Mujoco

In [5]:
def create_sim(xml_str, height, width, rgb=False, depth=False, segmentation=False):
    m = mujoco.MjModel.from_xml_string(xml_str)
    d = mujoco.MjData(m)

    res = [m, d]

    if rgb:
        r_rgb = mujoco.Renderer(m, height, width)
        res.append(r_rgb)

    if depth:
        r_depth = mujoco.Renderer(m, height, width)
        r_depth.enable_depth_rendering()
        res.append(r_depth)

    if segmentation:
        r_seg = mujoco.Renderer(m, height, width)
        r_seg.enable_segmentation_rendering()
        res.append(r_seg)

    return res


def new_xml(models, quat):  
    assets = ""
    geoms = ""

    for model in models:
        x = (np.random.random() - 0.5)
        y = (np.random.random() - 0.5) 
        z = (np.random.random() - 0.5) + 0.395
        assets += f'<mesh name="{model}"  file="ycb/{model}" scale="1 1 1"/>\n'
        geoms += f'<geom name="{model}" pos="{x} {y} {z}" type="mesh" contype="0" conaffinity="0" group="1" density="0" mesh="{model}" quat="{quat[0]} {quat[1]} {quat[2]} {quat[3]}"/>\n'

    xml =  f"""
    <mujoco>
        <visual>
            <quality offsamples="0"/>
        </visual>
        <asset>
            {assets}
        </asset>
        <worldbody>
            <light name="top" pos="0 0 1"/>
            {geoms}
                        
            <camera name="camera1" pos="0 -1 0.395" euler="90 0 0" mode="fixed" fovy="60" />
            <camera name="camera2" pos="1 0 0.395"  euler="90 90 0" mode="fixed" fovy="60" />
            <camera name="camera3" pos="0 1 0.395"  euler="90 180 0" mode="fixed" fovy="60" />
            <camera name="camera4" pos="-1 0 0.395" euler="90 270 0" mode="fixed" fovy="60" />

            <camera name="camera5" pos="0.75 -0.75 0.395"  euler="90 45 0" mode="fixed" fovy="60" />
            <camera name="camera6" pos="0.75 0.75 0.395"   euler="90 135 0" mode="fixed" fovy="60" />
            <camera name="camera7" pos="-0.75 0.75 0.395"  euler="90 225 0" mode="fixed" fovy="60" />
            <camera name="camera8" pos="-0.75 -0.75 0.395" euler="90 315 0" mode="fixed" fovy="60" />
              
        </worldbody>
    </mujoco>
    """

    return xml

def show_pc(pc):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc)
    o3d.visualization.draw_geometries([pcd])

def draw_pc(pc):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.vstack((pc, np.array([1,1,1]))))
    o3d.visualization.draw_plotly([pcd])

def transform_point_cloud(pc_array):
    pc_array = pc_array - np.mean(pc_array, axis=0)
    pc_array = pc_array[np.random.choice(pc_array.shape[0], 512, replace=True), :]
    pc_array = pc_array / np.max(np.linalg.norm(np.abs(pc_array), axis=1))
    return pc_array

In [6]:
# Generate xml for simulation with variables
quat = np.random.random(4)
quat = quat / np.linalg.norm(quat)
models = os.listdir('ycb')
stl_models = np.random.choice(models, size=5, replace=False)
# stl_models = np.append(models, ['073-g_lego_duplo.stl', '072-a_toy_airplane.stl', '062_dice.stl', '032_knife.stl'])
xml = new_xml(stl_models, quat)

# Instantiate the simulation
m, d, r_rgb, r_depth, r_seg = create_sim(xml, 480, 640, 
                                            rgb=True, 
                                            depth=True, 
                                            segmentation=True)

stl_models

array(['062_dice.stl', '026_sponge.stl', '063-a_marbles.stl',
       '077_rubiks_cube.stl', '073-e_lego_duplo.stl'], dtype='<U29')

In [7]:
# Obtaining extrinsic details of the cameras
cam_num = m.cam_user.shape[0]
cam_pos = []
cam_rot = []
for i in range(cam_num):
    cam_pos.append(m.cam_pos[i])
    cam_rot.append(R.from_quat(m.cam(i).quat).as_matrix())

# Instantiating the sensors and the pointcloud
sensor = Sensors()
point_cloud = VectorizedPC((480, 640), 60)
agents = {}

# Iterating trhought different time steps - t
for i in range(3):
    # Making first step for the simulation
    # mujoco.mj_step(m, d)

    # Get RGB, depth, and segmentation images
    depth = [*sensor.get_depth_image_matrices(m, d, r_depth).values()]
    segmn = [*sensor.get_segment_image_matrices(m, d, r_seg).values()]

    # Obtaining the pointclouds from the depth images segmented
    pc = {}
    cameras = [0,1,2,3,4,5,6,7]
    for i in cameras:
        rot = (R.from_matrix(cam_rot[i]).as_euler('xyz', degrees=True)[0] - 180) * -1
        rot = R.from_euler('xyz', [rot, 0, 90], degrees=True).as_matrix()
        aux = point_cloud.get_segmented_points(depth[i],
                                            segmn[i],
                                            rot,
                                            cam_pos[i])
        for key in aux.keys():
            if key == -1:
                continue
            try:
                pc[key] = np.concatenate((pc[key], aux[key]))
            except:
                pc[key] = aux[key]

    # Generating the agents
    for key in pc.keys():
        pos = np.average(pc[key], axis=0)
        try:
            agents[key].update_pos(pos)
        except:
            pc_array = transform_point_cloud(pc[key])
            feature_vector = get_feature_vector(transform_point_cloud(pc_array)).flatten()
            agents[key] = Agent(key, feature_vector, pos)
    
    # Moving the objects
    point_cloud.move_simple(t=1, func=lambda x: x * 0.001, axis=0, model=m)
    # Making first step for the simulation
    mujoco.mj_step(m, d)

In [8]:
# import open3d as o3d
# import numpy as np

# # Generate 1000 random points in a sphere
# points = np.random.rand(100, 3)
# # Repeat same color for 100 points
# colors = np.tile([255, 0, 0], (100, 1))

# # Generate another 1000 random points in a sphere
# points_2 = np.random.rand(100, 3)
# # Repeat same color for 100 points
# colors_2 = np.tile([0, 255, 0], (100, 1))

# # Concatenate the points and colors
# points = np.vstack((points, points_2))
# colors = np.vstack((colors, colors_2))

# # Create a point cloud from the points
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points)
# pcd.colors = o3d.utility.Vector3dVector(colors)

# o3d.visualization.draw_geometries([pcd])


In [9]:
for i in list(pc.keys()):
    # print(agents[i].vector().shape)
    pc_array = transform_point_cloud(pc[i])
    np.save(f'sanity_check_{i}.npy', pc_array)

## Label Generation

In [10]:
import os

names = os.listdir('ycb')

labels = {}

for name in names:
    id = int(name[:3])
    labels[id] = name


## Models Testing

In [11]:
# Load the model from pt files
model_paths = [path for path in os.listdir('models')]
model_paths = ['modelv9.pt']
models = []
for path in model_paths:
    model = Net()
    model.to('cuda')
    model.load_state_dict(torch.load('models/'+path)['model_state_dict'])
    model.eval()
    models.append(model)

for i in range(len(models)):
    for j in range(len(stl_models)):

        # Loading the pointcloud
        pc_load = np.load(f"sanity_check_{j}.npy")

        # Creating the data object
        data = Data(x = torch.tensor(pc_load, dtype=torch.float32)) 
        loader = DataLoader(dataset=[data], batch_size=1)

        # Obtaining the prediction
        for d in loader:
            d = d.to('cuda')
            result = models[i](d)[0].max(1)[1]

        print(f"{model_paths[i]} => ", result, " : ", labels[result.item()])
        
    print()

modelv9.pt =>  tensor([70], device='cuda:0')  :  070-b_colored_wood_blocks.stl
modelv9.pt =>  tensor([26], device='cuda:0')  :  026_sponge.stl
modelv9.pt =>  tensor([63], device='cuda:0')  :  063-a_marbles.stl
modelv9.pt =>  tensor([77], device='cuda:0')  :  077_rubiks_cube.stl
modelv9.pt =>  tensor([73], device='cuda:0')  :  073-g_lego_duplo.stl



In [12]:
for i in range(len(stl_models)):
    # Loading the pointcloud
    pc_load = np.load(f"sanity_check_{i}.npy")

    # Processing the information as in _datasets.py
    centroid = np.mean(pc_load, axis=0)
    pc_load = pc_load - centroid
    normalized = pc_load / np.std(pc_load)

    # Creating the data object
    data = Data(x = torch.tensor(pc_load, dtype=torch.float32)) 
    loader = DataLoader(dataset=[data], batch_size=1)
    for d in loader:
        pc = d.x
        draw_pc(pc)

### Prediction Debugging

In [13]:
model = Net()
model.to('cuda')
model.load_state_dict(torch.load('models/modelv3.pt')['model_state_dict'])
model.eval()


# Loading test dataset frompt files
test_dataset = ObjectPointCloudDataset(root = '.', 
                                    chunk = (87984, 109980), 
                                    sample_count = 512,
                                    output_name = 'testv9'
                                    )

loader = DataLoader(test_dataset,
                    batch_size=1)

point_clouds = []
label = []

test_num = 0
for d in loader:
    test_num += 1
    d = d.to('cuda')
    result = model(d)[0].max(1)[1]

    print(result, " : ", labels[result.item()], " | ", d.y)
    print(d.x.dtype)

    pc_cpu = d.x.cpu().numpy()

    point_clouds.append(pc_cpu)
    label.append(labels[result.item()])

    if test_num > 2:
        break

done loading
tensor([28], device='cuda:0')  :  028_skillet_lid.stl  |  tensor([73], device='cuda:0')
torch.float32
tensor([73], device='cuda:0')  :  073-g_lego_duplo.stl  |  tensor([72], device='cuda:0')
torch.float32
tensor([62], device='cuda:0')  :  062_dice.stl  |  tensor([62], device='cuda:0')
torch.float32


In [14]:
a = np.load('../dataset/v4/003_cracker_box_000003.npy')
show_pc(a)